# Combined Office and General Reviews:

In [1]:
# install transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.1 MB/s eta 0:00:00:00:0100:01


In [1]:
# load: 
import os 
import json
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# load the pre-trained models: 
from transformers import RobertaTokenizer, TFRobertaModel
with tf.device('/device:GPU:0'):
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = TFRobertaModel.from_pretrained("roberta-base")

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [3]:
os.getcwd()

'/home/ec2-user/SageMaker/Kartik_Amazon_Thesis/disagreement_with_rating'

In [4]:
# load csvs: 
# train_df = pd.read_csv('train_df_processed.csv')
# test_df =pd.read_csv('test_df_processed.csv')
train_path = '/home/ec2-user/SageMaker/Kartik_Amazon_Thesis/general_training_data'
disagreement_df = pd.read_csv(os.path.join(train_path,'disagreement_with_ratings_general.csv' ))
disagreement_df.columns

Index(['review_id', 'full_review', 'disagreement_with_ratings',
       'disagreement_with_ratings_ohe'],
      dtype='object')

In [5]:
disagreement_df.disagreement_with_ratings_ohe.value_counts()

0    10694
1      536
Name: disagreement_with_ratings_ohe, dtype: int64

In [6]:
disagreement_df[disagreement_df['disagreement_with_ratings_ohe']==1].iloc[500].full_review

"I usually never write reviews and I never write a negative one but.....I am so surprised that there are so many people saying this item actually works!! It doesn't! It's so flimsy. First of all putting it together is not that hard but it falls apart when you roll it! I can't even believe people can say it's great. It's not. Save yourself money and buy something that is sturdy and you can actually roll without it falling apart."

In [7]:
disagreement_df= disagreement_df[['disagreement_with_ratings_ohe', 'full_review']]

In [8]:
# train test split - split trainning into train and validation, take 10% as validation
train, test_df = train_test_split(disagreement_df, test_size= 0.3, random_state=42, stratify=disagreement_df['disagreement_with_ratings_ohe'])

In [9]:
# train test split - split trainning into train and validation, take 10% as validation
train_df_1, val_df = train_test_split(train, test_size= 0.2, random_state=42, stratify=train['disagreement_with_ratings_ohe'])

In [10]:
train_df_1.to_csv('train_dataset_disagreement_with_ratings.csv', index = False)

In [13]:
val_df.disagreement_with_ratings_ohe.value_counts()

0    1498
1      75
Name: disagreement_with_ratings_ohe, dtype: int64

In [14]:

class create_tensor_dataset(): 
    
    '''Purpose: Create a tensorflow dataset for model training
    df_to_convert: The dataframe of interest, that contains the 
    text columns
    text: The column within the dataframe containing the text 
    batch_size: int, batch size
    tokenizer: The tokenizer to be utilised
    labels: The class/label column within the dataframe
    '''
    
    def __init__(self, df_to_convert, text,target_col, batch_size, labels=None ): 
        self.df_to_convert = df_to_convert
        self.text = text # column containing the text (here chunked_data)
        self.target_col = target_col # y column of interest here label
        self.batch_size = batch_size
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.labels = labels # list of all the labels 
        
    def generate_training_data(self, ids, masks):
        for i, text in tqdm(enumerate(self.df_to_convert[self.text])):
            with tf.device('/device:GPU:0'):
                
                tokenized_text =  self.tokenizer.encode_plus(
                    text,# The sequence or batch of sequences to be encoded.
                    max_length=256, #Controls the maximum length to use by one of the truncation/padding parameters.
                    truncation=True, # Truncate to a maximum length specified with the argument max_length
                    padding='max_length', #pad to a maximum length specified with the argument max_length 
                    add_special_tokens=True,# encode the sequences with the special tokens relative to their model.
                    return_tensors='tf'   #return TensorFlow tf.constant objects.
                )
                ids[i, :] = tokenized_text.input_ids ##List of token ids to be fed to a model
                masks[i, :] = tokenized_text.attention_mask #List of indices specifying which tokens should be attended to by the model
        return ids, masks
    
    def ohe_label(self): 
         # convert label values to a list 
        return self.df_to_convert[self.target_col].values.tolist()

    def DatasetMapFunction(self, input_ids, attn_masks, labels):
        # create and return key-value pair of input ids, attention mask and the corresnponding labels
        return {
            'input_ids': input_ids,
            'attention_mask': attn_masks
        }, labels
    
    def tensor_data(self): 
        # input_ids array
        X_input_ids = np.zeros((len(self.df_to_convert), 256))
        # attention mask array
        X_attn_masks = np.zeros((len(self.df_to_convert), 256))
        # retrieve the input ids and attention masks
        X_input_ids, X_attn_masks = self.generate_training_data(X_input_ids, X_attn_masks)
        # creation of tensorflow dataset: 
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, self.ohe_label()))
        dataset = dataset.map(self.DatasetMapFunction)
        dataset = dataset.shuffle(10000).batch(self.batch_size, drop_remainder=True)
        return dataset

In [15]:
# creation of tensorflow dataset for training
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
train_tensor =create_tensor_dataset(train_df_1, 'full_review', 'disagreement_with_ratings_ohe',8 )
train_dataset =train_tensor.tensor_data() 

6288it [00:07, 836.61it/s]


In [16]:
# validation tensor dataset: 
with tf.device('/device:GPU:0'):
    val_tensor =create_tensor_dataset(val_df, 'full_review', 'disagreement_with_ratings_ohe', 8)
    val_dataset =val_tensor.tensor_data() 

1573it [00:02, 739.67it/s]


In [17]:

# defining 2 input layers for input_ids and attn_masks
with tf.device('/device:GPU:0'):
    input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
    attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
    bert_embds = model.roberta(input_ids, attention_mask=attn_masks)[1] # pooled output
    
    intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
   
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(intermediate_layer) # sigmoid -> calcs probs of binary classes
    model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
    model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [18]:
# utilization of adam optimization 
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
# utilised binary cross entropy loss 
loss_func = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])

# early stopping - administer val_loss: 
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience= 3,
    mode='min'
)

with tf.device('/device:GPU:0'):

    hist = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100, 
        callbacks =[custom_early_stopping]
    )

Epoch 1/100
786/786 [==============================] - 406s 464ms/step - loss: 0.1830 - accuracy: 0.9497 - val_loss: 0.1095 - val_accuracy: 0.9668
Epoch 2/100
786/786 [==============================] - 353s 449ms/step - loss: 0.0971 - accuracy: 0.9709 - val_loss: 0.0870 - val_accuracy: 0.9668
Epoch 3/100
786/786 [==============================] - 350s 446ms/step - loss: 0.0655 - accuracy: 0.9814 - val_loss: 0.0808 - val_accuracy: 0.9745
Epoch 4/100
786/786 [==============================] - 351s 446ms/step - loss: 0.0413 - accuracy: 0.9885 - val_loss: 0.0831 - val_accuracy: 0.9739
Epoch 5/100
786/786 [==============================] - 351s 446ms/step - loss: 0.0336 - accuracy: 0.9913 - val_loss: 0.1054 - val_accuracy: 0.9668
Epoch 6/100
786/786 [==============================] - 350s 446ms/step - loss: 0.0198 - accuracy: 0.9938 - val_loss: 0.1168 - val_accuracy: 0.9707


In [19]:
def prepare_data(input_text, tokenizer):
    with tf.device('/device:GPU:0'):
        
        token = tokenizer.encode_plus(
            input_text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        return {
            'input_ids': tf.cast(token.input_ids, tf.float64),
            'attention_mask': tf.cast(token.attention_mask, tf.float64)
        }

def make_predictions(model, input_text,threshold, label_list=None ): 
    with tf.device('/device:GPU:0'):
        processed_data = prepare_data(input_text, tokenizer)
        probs = model.predict(processed_data)
#         print(probs)
        if probs[0]> threshold: 
            return label_list[0]
        else: 
            return label_list[1]
#     return(probs[0])

In [20]:
# tt = test_df.head(20)
label_list= [1, 0]
test_df['result']= test_df.full_review.apply(lambda x: make_predictions(model, x, 0.5, label_list))

1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 43ms/step


...

1/1 [==============================] - 0s 50ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 44ms/step


In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['disagreement_with_ratings_ohe'], test_df['result']))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3208
           1       0.73      0.71      0.72       161

    accuracy                           0.97      3369
   macro avg       0.86      0.85      0.85      3369
weighted avg       0.97      0.97      0.97      3369



In [28]:
model.save("general_disagreement_with_ratings_model_fnl")

INFO:tensorflow:Assets written to: general_disagreement_with_ratings_model_fnl/assets


INFO:tensorflow:Assets written to: general_disagreement_with_ratings_model_fnl/assets


In [37]:
model_ = tf.keras.models.load_model(os.path.join(os.getcwd(),"general_disagreement_with_ratings_model_fnl"))

In [38]:
processed_data = prepare_data(a, tokenizer)
probs = model_.predict(processed_data)
print(probs)

1/1 [==============================] - 3s 3s/step
[[0.98820156]]


In [39]:
a = test_df[test_df.disagreement_with_ratings_ohe==1].full_review.iloc[10]

In [40]:
a

'They may only print two to three hundred then run out of life. Do not recommend buying it, doubt where the good reviews came from.'